# Assessment Tasks 

# Part 1: Data Loading and Cleaning 
# Download and Load Data 


# Download and Load Data
# ● Download the compressed dataset from the FTP server
# ● Handle the `.gz` compression format appropriately
# ● Load the data into a suitable Python data structure
# ● Implement proper error handling for network/file operations

# Step 1: Download and Extract the Data

In [6]:
import requests

url = "https://ita.ee.lbl.gov/traces/calgary_access_log.gz"
filename = "calgary_access_log.gz"

try:
    response = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print("Downloaded successfully.")
except Exception as e:
    print(f"Download failed: {e}")



Downloaded successfully.


# Data Cleaning
# ● Parse timestamp strings into `datetime` objects
# Confidential do not share https://mapup.ai/ 3

# ● Extract file extensions from the filename field
# ● Handle missing or malformed data entries
# ● Convert data types appropriately (integers, strings, etc.)
# ● Remove or flag invalid log entries

 # Step 2: Read and Decompress the File

In [7]:
log_lines = []

try:
    with gzip.open(filename, 'rt', encoding='utf-8', errors='ignore') as file:
        for line in file:
            log_lines.append(line.strip())
    print(f"Loaded {len(log_lines)} lines from the log file.")
except Exception as e:
    print(f"Error reading file: {e}")


Loaded 726739 lines from the log file.


# Step 3: Basic Cleaning and Structure

In [11]:
print(f"Total raw lines: {len(log_lines)}")
print(log_lines[0])  

Total raw lines: 726739
local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150


In [13]:
line = 'local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150'
print(line)

local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150


In [16]:
import re
from datetime import datetime

log_pattern = re.compile(
    r'(?P<host>\S+) \S+ \S+ \[(?P<timestamp>.+?)\] "(?P<request>.*?)" (?P<status>\d{3}) (?P<bytes>\S+)'
)

parsed_logs = []

for line in log_lines:
    match = log_pattern.match(line)
    if match:
        data = match.groupdict()
        
        try:
            data['timestamp'] = datetime.strptime(data['timestamp'], "%d/%b/%Y:%H:%M:%S %z")
        except Exception:
            continue  
        data['bytes'] = int(data['bytes']) if data['bytes'].isdigit() else 0
        request_parts = data['request'].split()
        if len(request_parts) == 3:
            data['method'], data['resource'], data['protocol'] = request_parts
        else:
            data['method'] = data['resource'] = data['protocol'] = None
        
        parsed_logs.append(data)
    else:
        continue  

 # Step 4: Convert to a DataFrame

In [17]:
import pandas as pd
df = pd.DataFrame(parsed_logs)

 # Step 5: Extract File Extensions

In [18]:
df['file_extension'] = df['resource'].apply(
    lambda x: x.split('.')[-1] if x and '.' in x else None
)

# Step 6: Final Cleaning

In [21]:
df_cleaned = df.dropna(subset=['timestamp', 'resource']).copy()
df_cleaned['status'] = df_cleaned['status'].astype(int) 


In [22]:
import pandas as pd
from collections import Counter, defaultdict
from datetime import datetime


# Q1. Total number of log records

In [23]:
total_logs = len(df_cleaned)
print(total_logs)


722341


# Q2. Count of unique hosts

In [24]:
unique_hosts = df_cleaned['host'].nunique()
print(unique_hosts)


2


# Q3. Date-wise unique filename counts

In [26]:
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'], errors='coerce')
df_cleaned = df_cleaned.dropna(subset=['timestamp'])
df_cleaned['date_str'] = df_cleaned['timestamp'].dt.strftime('%d-%b-%Y')


In [27]:
df_cleaned['date_str'] = df_cleaned['timestamp'].dt.strftime('%d-%b-%Y')
datewise_unique_files = (
    df_cleaned.groupby('date_str')['resource']
    .nunique()
    .to_dict()
)
print(datewise_unique_files)


{'01-Aug-1995': 669, '01-Jul-1995': 387, '01-Jun-1995': 590, '01-May-1995': 467, '01-Oct-1995': 552, '01-Sep-1995': 328, '02-Apr-1995': 438, '02-Aug-1995': 855, '02-Jul-1995': 397, '02-Jun-1995': 513, '02-May-1995': 701, '02-Oct-1995': 871, '02-Sep-1995': 349, '03-Apr-1995': 795, '03-Aug-1995': 582, '03-Jul-1995': 433, '03-Jun-1995': 398, '03-May-1995': 589, '03-Oct-1995': 846, '03-Sep-1995': 212, '04-Apr-1995': 821, '04-Aug-1995': 715, '04-Jul-1995': 610, '04-Jun-1995': 353, '04-May-1995': 684, '04-Oct-1995': 889, '04-Sep-1995': 340, '05-Apr-1995': 891, '05-Aug-1995': 507, '05-Jul-1995': 607, '05-Jun-1995': 494, '05-May-1995': 609, '05-Oct-1995': 846, '05-Sep-1995': 411, '06-Apr-1995': 678, '06-Aug-1995': 448, '06-Jul-1995': 522, '06-Jun-1995': 662, '06-May-1995': 517, '06-Oct-1995': 868, '06-Sep-1995': 549, '07-Apr-1995': 776, '07-Aug-1995': 608, '07-Jul-1995': 428, '07-Jun-1995': 486, '07-May-1995': 725, '07-Oct-1995': 468, '07-Sep-1995': 590, '08-Apr-1995': 542, '08-Aug-1995': 654,

# Q4. Number of 404 response codes

In [28]:
not_found_404 = (df_cleaned['status'] == 404).sum()
print(not_found_404)


14586


# Q5. Top 15 filenames with 404

In [29]:
top_404_files = (
    df_cleaned[df_cleaned['status'] == 404]['resource']
    .value_counts()
    .head(15)
    .items()
)
print(list(top_404_files))


[('index.html', 3119), ('4115.html', 901), ('1611.html', 647), ('5698.xbm', 500), ('710.txt', 254), ('10695.ps', 161), ('6555.html', 153), ('9678.gif', 142), ('3268.gif', 138), ('9814.gif', 134), ('11059.gif', 129), ('11060.gif', 129), ('9388.xbm', 120), ('151.html', 119), ('1685.html', 113)]


# Q6. Top 15 file extensions with 404

In [30]:
df_cleaned['file_extension'] = df_cleaned['resource'].apply(
    lambda x: x.split('.')[-1] if x and '.' in x else 'none'
)

top_404_ext = (
    df_cleaned[df_cleaned['status'] == 404]['file_extension']
    .value_counts()
    .head(15)
    .items()
)
print(list(top_404_ext))


[('html', 8051), ('gif', 4013), ('xbm', 665), ('ps', 562), ('txt', 265), ('jpg', 200), ('cgi', 76), ('GIF', 42), ('htm', 40), ('gif"', 34), ('com', 29), ('com/', 24), ('dvi', 23), ('rgb', 21), ('html/', 21)]


# Q7. Total bandwidth transferred per day for July 1995

In [31]:
df_july95 = df_cleaned[
    (df_cleaned['timestamp'].dt.month == 7) &
    (df_cleaned['timestamp'].dt.year == 1995)
]

df_july95['date_str'] = df_july95['timestamp'].dt.strftime('%d-%b-%Y')

bandwidth_per_day = (
    df_july95.groupby('date_str')['bytes']
    .sum()
    .to_dict()
)
print(bandwidth_per_day)


{'01-Jul-1995': 11333976, '02-Jul-1995': 8653986, '03-Jul-1995': 13508529, '04-Jul-1995': 26565884, '05-Jul-1995': 19541225, '06-Jul-1995': 19752989, '07-Jul-1995': 9427822, '08-Jul-1995': 5403491, '09-Jul-1995': 4660556, '10-Jul-1995': 14912796, '11-Jul-1995': 22503471, '12-Jul-1995': 17365039, '13-Jul-1995': 15986302, '14-Jul-1995': 19184404, '15-Jul-1995': 15769181, '16-Jul-1995': 9005564, '17-Jul-1995': 19596435, '18-Jul-1995': 17096829, '19-Jul-1995': 17847673, '20-Jul-1995': 20751717, '21-Jul-1995': 25455607, '22-Jul-1995': 8059932, '23-Jul-1995': 9577795, '24-Jul-1995': 22298075, '25-Jul-1995': 24472760, '26-Jul-1995': 24564950, '27-Jul-1995': 25967969, '28-Jul-1995': 36456855, '29-Jul-1995': 11684209, '30-Jul-1995': 23158170, '31-Jul-1995': 30715614}


C:\Users\anant\AppData\Local\Temp\ipykernel_4312\181124977.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_july95['date_str'] = df_july95['timestamp'].dt.strftime('%d-%b-%Y')


# Q8. Hourly request distribution

In [32]:
df_cleaned['hour'] = df_cleaned['timestamp'].dt.hour
hourly_dist = df_cleaned['hour'].value_counts().sort_index().to_dict()
print(hourly_dist)


{0: 11510, 1: 9832, 2: 9346, 3: 8101, 4: 7789, 5: 8234, 6: 9750, 7: 11896, 8: 17302, 9: 21637, 10: 25627, 11: 28584, 12: 26749, 13: 29997, 14: 29636, 15: 28041, 16: 28202, 17: 23229, 18: 17778, 19: 17253, 20: 17437, 21: 15915, 22: 14500, 23: 13534}


# Q9. Top 10 most requested filenames

In [33]:
top_requested_files = (
    df_cleaned['resource']
    .value_counts()
    .head(10)
    .items()
)
print(list(top_requested_files))


[('index.html', 75299), ('3.gif', 11949), ('2.gif', 11559), ('4097.gif', 4733), ('8870.jpg', 4492), ('244.gif', 4339), ('6733.gif', 4278), ('8472.gif', 3843), ('8308.gif', 3478), ('4.gif', 3357)]


# Q10. HTTP response code distribution

In [34]:
status_distribution = df_cleaned['status'].value_counts().sort_index().to_dict()
print(status_distribution)


{200: 328438, 302: 16595, 304: 70131, 400: 7, 401: 46, 403: 2022, 404: 14586, 500: 28, 501: 26}
